In [15]:
from PIL import Image
import glob
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras

## 모델 변수 설정

In [77]:
# 리사이즈 변수
resize_num = 112

# 모델 설계 변수 설정
Conv2D_1 = 64
Conv2D_2 = 128
Conv2D_3 = 512
dense = 256

kernel_size = (3,3)
maxpool_size = (2,2)

# 모델 학습 변수 설정
epochs_nums = 20

### 파일 리사이즈 하는 함수

In [53]:
 # 파일 리사이즈 하는 함수
def resize_images(img_path, resize_number):
    images=glob.glob(img_path + "/*.jpg")  
    
    print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
    target_size=(resize_number, resize_number)
    save_path = img_path + "/resize" + str(resize_number)
    
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(save_path + "/" + img.split("/")[-1], "JPEG")
    
    print(len(images), " images resized.")
    
    return len(images)

# /aiffel/바꿀폴더/scissor or rock or paper 바꾸기
image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/scissor"
image_count = 0;

image_count += resize_images(image_dir_path, resize_num)
print("가위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/rock"
image_count += resize_images(image_dir_path, resize_num)
print("바위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/paper"
image_count += resize_images(image_dir_path, resize_num)
print("보 이미지 resize 완료!")

print(image_count)

# test set
image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/test/scissor"
image_count = 0;

image_count += resize_images(image_dir_path, resize_num)
print("가위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/test/rock"
image_count += resize_images(image_dir_path, resize_num)
print("바위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/test/paper"
image_count += resize_images(image_dir_path, resize_num)
print("보 이미지 resize 완료!")

print(image_count)

100  images to be resized.
100  images resized.
가위 이미지 resize 완료!
100  images to be resized.
100  images resized.
바위 이미지 resize 완료!
100  images to be resized.
100  images resized.
보 이미지 resize 완료!
300
100  images to be resized.
100  images resized.
가위 이미지 resize 완료!
100  images to be resized.
100  images resized.
바위 이미지 resize 완료!
100  images to be resized.
100  images resized.
보 이미지 resize 완료!
300


### 이미지 부풀리기

In [54]:
def image_generator (img_path):
    
    save_path = img_path
    images=glob.glob(img_path + "/*.jpg")
        
    count = 0
    
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.transpose(Image.FLIP_LEFT_RIGHT)
        new_img.save(save_path + "/" + str(len(images)) + str(count) + ".jpg", "JPEG")
        count += 1
        
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.transpose(Image.FLIP_TOP_BOTTOM)
        new_img.save(save_path + "/" + str(len(images)) + str(count) + ".jpg", "JPEG")
        count += 1


# 이미지 경로
image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/scissor/resize112"
image_generator(image_dir_path)
print("가위 부풀리기 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/rock/resize112"
image_generator(image_dir_path)
print("바위 부풀리기 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/paper/resize112"
image_generator(image_dir_path)
print("보 부풀리기 완료!")

가위 부풀리기 완료!
바위 부풀리기 완료!
보 부풀리기 완료!
300


In [60]:
image_count = 0;

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/scissor/resize112"
images=glob.glob(image_dir_path + "/*.jpg")
image_count += len(images)

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/rock/resize112"
images=glob.glob(image_dir_path + "/*.jpg")
image_count += len(images)

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/paper/resize112"
images=glob.glob(image_dir_path + "/*.jpg")
image_count += len(images)

print(image_count)

900


### 이미지 라벨링 하는 함수
### 가위 0, 바위 1, 보 2

In [61]:
# 이미지 라벨링 하는 함수
def load_data(img_path, number_of_data, param_img_size):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=param_img_size
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    print(img_path + '/scissor/resize' + str(img_size) + '/*.jpg')
    
    idx=0
    for file in glob.iglob(img_path + '/scissor/resize' + str(img_size) + '/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/resize' + str(img_size) + '/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/resize' + str(img_size) + '/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

# /aiffel/바꿔야하는 폴더
image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp"
(x_train, y_train)=load_data(image_dir_path,image_count, resize_num)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

/aiffel/aiffel/aiffel_rsp/scissor/resize112/*.jpg
학습데이터(x_train)의 이미지 개수는 900 입니다.
x_train shape: (900, 112, 112, 3)
y_train shape: (900,)


### 모델 설계

In [79]:
# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(Conv2D_1, kernel_size, activation='relu', input_shape=(resize_num,resize_num,3)))
model.add(keras.layers.MaxPool2D(maxpool_size))
model.add(keras.layers.Conv2D(Conv2D_2, kernel_size, activation='relu'))
model.add(keras.layers.MaxPooling2D(maxpool_size))
model.add(keras.layers.Conv2D(Conv2D_3, kernel_size, activation='relu'))
model.add(keras.layers.MaxPooling2D(maxpool_size))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

# 모델 요약
model.summary()

Model에 추가된 Layer 개수:  9
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 110, 110, 64)      1792      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 53, 53, 128)       73856     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 24, 24, 512)       590336    
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 12, 12, 512)       0         
_________________________________________________________________
flatten_9 (Flatten)          (

### 모델 학습

In [80]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.compile()과 model.fit()을 사용해 봅시다.
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=epochs_nums)

Epoch 1/20
29/29 [==============================] - 2s 35ms/step - loss: 62.7510 - accuracy: 0.3233
Epoch 2/20
29/29 [==============================] - 1s 28ms/step - loss: 1.0939 - accuracy: 0.3500
Epoch 3/20
29/29 [==============================] - 1s 28ms/step - loss: 1.0695 - accuracy: 0.4667
Epoch 4/20
29/29 [==============================] - 1s 28ms/step - loss: 1.0207 - accuracy: 0.5467
Epoch 5/20
29/29 [==============================] - 1s 28ms/step - loss: 0.9812 - accuracy: 0.5100
Epoch 6/20
29/29 [==============================] - 1s 28ms/step - loss: 0.8568 - accuracy: 0.5889
Epoch 7/20
29/29 [==============================] - 1s 28ms/step - loss: 0.7215 - accuracy: 0.6700
Epoch 8/20
29/29 [==============================] - 1s 28ms/step - loss: 0.6175 - accuracy: 0.7467
Epoch 9/20
29/29 [==============================] - 1s 28ms/step - loss: 0.6618 - accuracy: 0.7300
Epoch 10/20
29/29 [==============================] - 1s 28ms/step - loss: 0.5294 - accuracy: 0.7900
Epoch 11

### 트레이닝 셋 만들기

In [68]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.

image_dir_path = os.getenv("HOME") + "/aiffel/aiffel_rsp/test"
print(image_dir_path)
(x_test, y_test)=load_data(image_dir_path, 300, resize_num)
x_test_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

/aiffel/aiffel/aiffel_rsp/test
/aiffel/aiffel/aiffel_rsp/test/scissor/resize112/*.jpg
학습데이터(x_train)의 이미지 개수는 300 입니다.
x_test shape: (300, 112, 112, 3)
y_test shape: (300,)


### 테스트 결과

In [81]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.evaluate()을 사용해 봅시다.
test_loss, test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 3.0801 - accuracy: 0.7600
test_loss: 3.0801265239715576 
test_accuracy: 0.7599999904632568


### 테스트 기록

### 1. 이미지 개수 300, 이미지 사이즈 28, Conv2D(64,128,256), Dense(128), epoch=10
train accuracy = 0.84
test accuracy : 0.54

### 2. 이미지 개수 300, 이미지 사이즈 56, Conv2D(64,128,256), Dense(128), epoch=10
train accuracy = 0.99
test accuracy : 0.43

### 3. 이미지 개수 300, 이미지 사이즈 112, Conv2D(64,128,256), Dense(128), epoch=10
train accuracy = 0.72
test accuracy : 0.31

이미지가 커지면서 정확도가 증가할줄 알았는데, 사이즈를 늘릴때 줄어든거보니 epoch이 부족했던건 아닐까 생각이 들어 늘려보았다

### 4. 이미지 개수 300, 이미지 사이즈 112, Conv2D(64,128,256), Dense(128), epoch=15
train accuracy = 0.98
test accuracy : 0.36

### 5. 이미지 개수 300, 이미지 사이즈 112, Conv2D(64,128,256), Dense(128), epoch=20
train accuracy = 1
test accuracy : 0.37

학습을 하는 중간에 1을 달성하는것을 보니 무작정 epoch를 늘리는것은 또 좋지 않은듯하다.
이미지 사이즈 112, epoch는 20으로 고정하고 하이퍼 파라미터를 늘려보았다.

### 6. 이미지 개수 300, 이미지 사이즈 112, Conv2D(128,256,512), Dense(128), epoch=20
train accuracy = 0.98
test accuracy : 0.45

### 7. 이미지 개수 300, 이미지 사이즈 112, Conv2D(128,256,512), Dense(256), epoch=20
train accuracy = 0.97
test accuracy : 0.51

커널 사이즈를 변경해볼까?
kernel_size = (3,3) -> (4,4)
maxpool_size = (2,2) -> (3,3)

### 8. 이미지 개수 300, 이미지 사이즈 112, Conv2D(128,256,512), Dense(256), epoch=20
train accuracy = 0.97
test accuracy : 0.37

커널 사이즈 변경은 아직인듯 하다. 다시 kernel_size = (3,3) maxpool_size = (2,2)으로 변경
이미지의 갯수가 좀 부족한가?
Image의 내장함수인 transpose로 좌우, 상하 반전을 시켜서 이미지 증가

### 9. 이미지 개수 900, 이미지 사이즈 112, Conv2D(128,256,512), Dense(256), epoch=20
train accuracy = 0.97
test accuracy : 0.75